https://mlang.frama.io/cours-marc-lang/stable/sigmaM2_telea/sigmaM2_projet.html#construire-un-masque-%C3%A0-partir-de-la-bd-for%C3%AAt

# **Construire un masque à partir de la BDFORET**


### Objectif
Le but est de produire un masque raster qui indique les zones à classer en forêt et les zones à ne pas classer, à partir des polygones de la **BD forêt**.

### Critères de classification
- **Zones à classer** : Ce sont les polygones de la BD forêt, à l'exception de :
  - **Lande** et **Formation Herbacée** (qui ne sont pas de la forêt),
  - **Forêt ouverte**,
  - **Forêt fermée sans couvert arboré**.
  
### Paramètres du masque
Le masque doit être conforme aux spécifications suivantes :

- **Format** : GeoTIFF
- **Encodage** : 8 bits
- **Emprise et résolution spatiale** : Doit correspondre à celles des images **S2** utilisées (après découpe, cf. section Pré-traitement des images)
- **Nom du fichier** : `masque_foret.tif`
- **Chemin de sortie** : `results/data/img_pretraitees`
- **Valeurs du masque** :
  - **Zone de forêt** : 1
  - **Zone hors forêt** : 0

### Exemple de valeurs dans le fichier raster
| Classe            | Valeur du pixel |
|-------------------|-----------------|
| Zone de forêt     | 1               |
| Zone hors forêt   | 0               |

In [214]:
shapefile_path = '../../data/project/FORMATION_VEGETALE.shp'
s2_image_path = '../../data/images/SENTINEL2B_20220326-105856-076_L2A_T31TCJ_C_V3-0_FRE_B11.tif'  # Exemple d'image Sentinel-2
output_path = '../results/data/img_pretraitees/masque_foret.tif'

In [215]:
import geopandas as gpd
from osgeo import gdal, ogr
import numpy as np

In [216]:
# Charger le shapefile
gdf = gpd.read_file(shapefile_path)

# Afficher les premières lignes de la table attributaire
print(gdf.head())

                         ID CODE_TFV                             TFV  \
0  FORESTIE0000000003100001      FO1  Forêt ouverte de feuillus purs   
1  FORESTIE0000000003100002      FO1  Forêt ouverte de feuillus purs   
2  FORESTIE0000000003100003      FO1  Forêt ouverte de feuillus purs   
3  FORESTIE0000000003100004      FO1  Forêt ouverte de feuillus purs   
4  FORESTIE0000000003100005      FO1  Forêt ouverte de feuillus purs   

                  TFV_G11   ESSENCE  \
0  Forêt ouverte feuillus  Feuillus   
1  Forêt ouverte feuillus  Feuillus   
2  Forêt ouverte feuillus  Feuillus   
3  Forêt ouverte feuillus  Feuillus   
4  Forêt ouverte feuillus  Feuillus   

                                            geometry  
0  POLYGON ((508756.27 6181000.546, 508761.718 61...  
1  POLYGON ((504437.854 6181272.919, 504437.135 6...  
2  POLYGON ((508679 6181464, 508680.8 6181463, 50...  
3  POLYGON ((509217.516 6181694.51, 509213.5 6181...  
4  POLYGON ((508056 6181920, 508053.3 6181915, 50...  


In [217]:
# Définir les valeurs à exclure
excluded_values = ['Formation herbacée' , 'Forêt ouverte de conifères purs' , 'Forêt ouverte de feuillus purs' , 'Forêt ouverte sans couvert arboré' , 'Forêt ouverte à mélange de feuillus et conifères' , 'Lande']

# Ajouter la colonne "mask" avec la condition spécifiée
gdf['mask'] = gdf['TFV'].apply(lambda x: 0 if x in excluded_values else 1)

# Afficher les premières lignes pour vérifier
print(gdf.head())

                         ID CODE_TFV                             TFV  \
0  FORESTIE0000000003100001      FO1  Forêt ouverte de feuillus purs   
1  FORESTIE0000000003100002      FO1  Forêt ouverte de feuillus purs   
2  FORESTIE0000000003100003      FO1  Forêt ouverte de feuillus purs   
3  FORESTIE0000000003100004      FO1  Forêt ouverte de feuillus purs   
4  FORESTIE0000000003100005      FO1  Forêt ouverte de feuillus purs   

                  TFV_G11   ESSENCE  \
0  Forêt ouverte feuillus  Feuillus   
1  Forêt ouverte feuillus  Feuillus   
2  Forêt ouverte feuillus  Feuillus   
3  Forêt ouverte feuillus  Feuillus   
4  Forêt ouverte feuillus  Feuillus   

                                            geometry  mask  
0  POLYGON ((508756.27 6181000.546, 508761.718 61...     0  
1  POLYGON ((504437.854 6181272.919, 504437.135 6...     0  
2  POLYGON ((508679 6181464, 508680.8 6181463, 50...     0  
3  POLYGON ((509217.516 6181694.51, 509213.5 6181...     0  
4  POLYGON ((508056 6181920

In [218]:
image_s2 = gdal.Open(s2_image_path)

In [219]:
# Obtenir la projection et l'emprise du raster
projection = image_s2.GetProjection()
geotransform = image_s2.GetGeoTransform()

print(geotransform)

raster_minx = geotransform[0]
raster_maxy = geotransform[3]
raster_resolution_x = geotransform[1]
raster_resolution_y = -geotransform[5]
raster_maxx = raster_minx + raster_resolution_x * image_s2.RasterXSize
raster_miny = raster_maxy + raster_resolution_y * image_s2.RasterYSize

(300000.0, 20.0, 0.0, 4900020.0, 0.0, -20.0)


In [220]:
gdf = gdf.to_crs(projection)

print(gdf.head())

                         ID CODE_TFV                             TFV  \
0  FORESTIE0000000003100001      FO1  Forêt ouverte de feuillus purs   
1  FORESTIE0000000003100002      FO1  Forêt ouverte de feuillus purs   
2  FORESTIE0000000003100003      FO1  Forêt ouverte de feuillus purs   
3  FORESTIE0000000003100004      FO1  Forêt ouverte de feuillus purs   
4  FORESTIE0000000003100005      FO1  Forêt ouverte de feuillus purs   

                  TFV_G11   ESSENCE  \
0  Forêt ouverte feuillus  Feuillus   
1  Forêt ouverte feuillus  Feuillus   
2  Forêt ouverte feuillus  Feuillus   
3  Forêt ouverte feuillus  Feuillus   
4  Forêt ouverte feuillus  Feuillus   

                                            geometry  mask  
0  POLYGON ((309032.524 4730491.765, 309037.965 4...     0  
1  POLYGON ((304718.567 4730755.446, 304717.85 47...     0  
2  POLYGON ((308954.455 4730954.532, 308956.254 4...     0  
3  POLYGON ((309491.904 4731185.805, 309487.913 4...     0  
4  POLYGON ((308331.3 47314

In [221]:
# Calculer la largeur et la hauteur du raster de sortie à partir de l'emprise et de la résolution
width = abs(int((raster_maxx - raster_minx) / raster_resolution_x))
height = abs(int((raster_maxy - raster_miny) / raster_resolution_y))

# Créer un raster vide en utilisant GDAL
driver = gdal.GetDriverByName('GTiff')
out_raster = driver.Create(output_path, width, height, 1, gdal.GDT_Byte)

# Définir la géotransformation et la projection pour le raster de sortie
out_raster.SetGeoTransform(geotransform)
out_raster.SetProjection(projection)

# Créer une couche pour la rasterisation
out_band = out_raster.GetRasterBand(1)
out_band.SetNoDataValue(0)  # Définir la valeur "no data"

# Initialiser le tableau avec des zéros (fond)
raster_data = np.zeros((height, width), dtype=np.uint8)

In [222]:
# Créer une couche mémoire OGR pour la rasterisation
mem_driver = ogr.GetDriverByName('Memory')
mem_ds = mem_driver.CreateDataSource('memory')
layer = mem_ds.CreateLayer('layer', geom_type=ogr.wkbPolygon)

# Ajouter un champ 'mask' à la couche mémoire
field_defn = ogr.FieldDefn('mask', ogr.OFTInteger)
layer.CreateField(field_defn)

# Ajouter les géométries et les valeurs du champ 'mask' à la couche mémoire
for _, row in gdf.iterrows():
    geometry = row['geometry']
    mask_value = row['mask']  # Utiliser la valeur du champ 'mask' pour la valeur du pixel
    
    ogr_geom = ogr.CreateGeometryFromWkt(geometry.wkt)
    feature = ogr.Feature(layer.GetLayerDefn())
    feature.SetGeometry(ogr_geom)
    feature.SetField('mask', mask_value)  # Assigner la valeur de 'mask' au champ 'mask'
    layer.CreateFeature(feature)

# Rasteriser la couche mémoire en utilisant la valeur du champ 'mask' pour chaque géométrie
gdal.RasterizeLayer(out_raster, [1], layer, options=["ATTRIBUTE=mask"])

# Sauvegarder et fermer le raster
out_raster.FlushCache()
del out_raster

# Afficher un message
print(f"Rasterisation terminée et sauvegardée dans '{output_path}'")

/opt/conda/lib/python3.12/site-packages/osgeo/gdal.py:8292: RuntimeWarning: Failed to fetch spatial reference on layer layer to build transformer, assuming matching coordinate systems.
  return _gdal.RasterizeLayer(*args, **kwargs)


Rasterisation terminée et sauvegardée dans '../results/data/img_pretraitees/masque_foret.tif'
